In [7]:
import pandas as pd
import json

# Load the CSV file into a DataFrame
data = pd.read_csv('data/formalizacion_data.csv')

# Load the JSON files
with open('data/mm_yyyy_sampled_files.json') as f:
    sampled_files = json.load(f)

with open('data/mm_yyyy_size.json') as f:
    size_data = json.load(f)

with open('data/mm_yyyy_weight.json') as f:
    weight_data = json.load(f)


# Data Cleaning

In [8]:
# Replace 'None' with NaN
data.replace('None', pd.NA, inplace=True)

# Drop rows where 'amount' is NaN
cleaned_data = data.dropna(subset=['amount'])

# Convert 'amount' and 'time' to numeric
cleaned_data['amount'] = pd.to_numeric(cleaned_data['amount'], errors='coerce')
cleaned_data['time'] = pd.to_numeric(cleaned_data['time'], errors='coerce')

# Convert 'mm_yyyy' to datetime
cleaned_data['mm_yyyy'] = pd.to_datetime(cleaned_data['mm_yyyy'], format='%m-%Y', errors='coerce')

/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/2711408758.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['amount'] = pd.to_numeric(cleaned_data['amount'], errors='coerce')
/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/2711408758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['time'] = pd.to_numeric(cleaned_data['time'], errors='coerce')
/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/2711408758.py:12: SettingWithCopyWarni

In [9]:
contract_mapping = {
    'Suministro': 'Contrato de Suministros',
    'Servicios': 'Contrato de Servicios',
    'Servicio': 'Contrato de Servicios',
    'Obras': 'Contrato de Obras',
    'Suministros': 'Contrato de Suministros',
    'Contrato de servicios': 'Contrato de Servicios',
    'Contrato de obras': 'Contrato de Obras',
    'Contrato de Servicios': 'Contrato de Servicios',
    'Gestión de servicio público': None,
    'Gestión de Servicios Públicos': None,
    'Ordinario': None,  # No es un tipo de contrato
    'Obra': 'Contrato de Obras',
    'Contrato administrativo de servicio': None,
    'Administrativo de servicio': None,  # No es un tipo de contrato, aunque luce de servicios
    'Contrato privado': None,  # No aplica a contratos del sector público
    'Administrativo': None,  # No específico
    'Suministro sucesivo y continuado': None, # No es un tipo de contrato
    'Privado': None,  # No aplica a contratos del sector público
    'Concesión del servicio público educativo': None # No es un tipo de contrato aunque luce de servicios
}

def correct_contract_types(df, mapping=contract_mapping):
    df['tipo'] = df['tipo'].replace(mapping)
    return df

# Apply the corrections
formalizacion_data_corrected = correct_contract_types(cleaned_data)

cleaned_data[cleaned_data["tramitacion"] == "Anticipada"]

tramitacion_mapping = {
    'Ordinaria': 'Ordinaria',
    'Urgente': 'Urgente',
    'Anticipada': 'Ordinaria',
    'Anticipada plurianual': 'Ordinaria',
    'Normal': 'Ordinaria',
    'Abierto': None,  # No es una tramitación
    'Ordinaria-Plurianual de gasto': 'Ordinaria'
}

# Aplicar el mapeo
cleaned_data['tramitacion'] = cleaned_data['tramitacion'].map(lambda x: tramitacion_mapping.get(x, x))

# Eliminar filas con tipos None (no agrupados)
cleaned_data = cleaned_data.dropna(subset=['tramitacion'])

# Supongamos que 'cleaned_data' es tu DataFrame

# Definir el diccionario de mapeo
procedimiento_mapping = {
    'Abierto': 'Procedimiento Abierto',
    'Negociado sin publicidad': 'Procedimiento Negociado',
    'Abierto sujeto a regulación armonizada': 'Procedimiento Abierto',
    'Abierto armonizado': 'Procedimiento Abierto',
    'Negociado sujeto a regulación armonizada': 'Procedimiento Negociado',
    'Abierto con varios criterios de adjudicación': 'Procedimiento Abierto',
    'Procedimiento negociado sin publicidad': 'Procedimiento Negociado',
    'Negociado sin publicidad sujeto a Regulación Armonizada': 'Procedimiento Negociado',
    'Abierto - Criterio precio': 'Procedimiento Abierto',
    'Abierto pluralidad de criterios': 'Procedimiento Abierto',
    'Simplificado': 'Procedimiento Simplificado',
    'Abierto con pluralidad de criterios': 'Procedimiento Abierto',
    'Abierto y un único criterio de adjudicación': 'Procedimiento Abierto',
    'Basado en un Acuerdo Marco': 'Procedimiento Abierto',
    'Abierto (sujeto a regulación armonizada)': 'Procedimiento Abierto',
    'Negociado sin publicidad artículo 170 d': 'Procedimiento Negociado',
    'Sin Publicidad': None,
    'Abierto - Varios criterios de adjudicación': 'Procedimiento Abierto',
    'Negociado Sin Publicidad': 'Procedimiento Negociado',
    'Abierto mediante criterio único precio': 'Procedimiento Abierto',
    'Abierto criterios múltiples': 'Procedimiento Abierto',
    'Abierto - Criterio Precio': 'Procedimiento Abierto',
    'Abierto Regulación Armonizada': 'Procedimiento Abierto',
    'Menor d.a. 9a TRLCSP': None,
    'Abierto - criterio precio': 'Procedimiento Abierto',
    'Abierto - Pluralidad de criterios': 'Procedimiento Abierto',
    'Abierto y sujeto a regulación harmonizada': 'Procedimiento Abierto',
    'Abierto criterio precio': 'Procedimiento Abierto',
    'Abierto – Criterio Precio': 'Procedimiento Abierto',
    'Abierto y regulación armonizada': 'Procedimiento Abierto',
    'Abierto sujeto a regulación armonizada con multiplicidad de criterios de adjudicación': 'Procedimiento Abierto',
    'Abierto con un único criterio': 'Procedimiento Abierto',
    'Derivado de Acuerdo Marco': 'Procedimiento Abierto',
    'Abierto de regulación armonizada': 'Procedimiento Abierto',
    'Abierto con criterio precio': 'Procedimiento Abierto',
    'Abierto varios criterios de adjudicación': 'Procedimiento Abierto',
    'Negociado sin Publicidad': 'Procedimiento Negociado',
    'Negociado con publicidad': 'Procedimiento Negociado',
    'Negociado sin publicidad con exclusividad': 'Procedimiento Negociado',
    'Negociado con publicidad y sujeto a regulación armonizada': 'Procedimiento Negociado',
    'Negociado': 'Procedimiento Negociado',
    'Abierto en base a varios criterios': 'Procedimiento Abierto',
    'Abierto/Pluralidad de criterios': 'Procedimiento Abierto',
    'Contrato sujeto a la regulación armonizada': 'Procedimiento Abierto',
    'Abierto multiplicidad de criterios': 'Procedimiento Abierto',
    'Abierto con un solo criterio cuantificable de forma automática': 'Procedimiento Abierto',
    'Abierto y pluralidad de criterios de adjudicación': 'Procedimiento Abierto'
}

# Aplicar el mapeo
cleaned_data['procedimiento'] = cleaned_data['procedimiento'].map(lambda x: procedimiento_mapping.get(x, x))

# Eliminar filas con tipos None (no agrupados)
cleaned_data = cleaned_data.dropna(subset=['procedimiento'])

currency_mapping = {
    'EUR': 'EUR',
    'euros': 'EUR',
    'Euros': 'EUR',
    '€': 'EUR'
}

# Aplicar el mapeo
cleaned_data['currency'] = cleaned_data['currency'].map(lambda x: currency_mapping.get(x, x))

# Eliminar filas con tipos None (no agrupados)
cleaned_data = cleaned_data.dropna(subset=['currency'])

import pandas as pd

def levenshtein_distance(a, b):
    if len(a) < len(b):
        return levenshtein_distance(b, a)
    
    if len(b) == 0:
        return len(a)
    
    previous_row = range(len(b) + 1)
    for i, c1 in enumerate(a):
        current_row = [i + 1]
        for j, c2 in enumerate(b):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def normalize_company_name(name):
    return str(name).strip().lower()

def calculate_frequencies(df, column_name='adjudicadora'):
    normalized_names = df[column_name].apply(normalize_company_name)
    unique_names = normalized_names.unique()
    
    groups = {}
    
    for target_name in unique_names:
        if target_name not in groups:
            groups[target_name] = []
        for name in unique_names:
            if levenshtein_distance(name, target_name) <= 3:
                groups[target_name].append(name)
    
    return groups

def standardize_names(df, column_name='adjudicadora'):
    groups = calculate_frequencies(df, column_name)
    
    standardization_map = {}
    
    for target_name, similar_names in groups.items():
        shortest_name = min(similar_names, key=len)
        for name in similar_names:
            standardization_map[name] = shortest_name
    
    df[column_name] = df[column_name].apply(normalize_company_name).map(standardization_map)
    return df

cleaned_data = standardize_names(cleaned_data, column_name='adjudicadora')
# New column with the name of the adjudicadora if it appears more than 3 times
cleaned_data['adjudicadora_grouped'] = cleaned_data['adjudicadora'].where(cleaned_data['adjudicadora'].map(cleaned_data['adjudicadora'].value_counts()) > 4, 'Otros')

cleaned_data = standardize_names(cleaned_data, column_name='company')


/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/245791945.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo'] = df['tipo'].replace(mapping)
/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/245791945.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['tramitacion'] = cleaned_data['tramitacion'].map(lambda x: tramitacion_mapping.get(x, x))
/var/folders/n4/s8124s797fjd6l5zb_sn00cw0000gn/T/ipykernel_5113/245791945.py:102: SettingWithCopyWarning: 
A value

In [13]:
# Save the cleaned data to a new CSV file
cleaned_data.to_csv('data/formalizacion_data_cleaned.csv', index=False)

In [17]:
import numpy as np
def proportion_ci_by_month(proportion, total, Z=1.96):
    return proportion.apply(lambda p: (p - Z * np.sqrt(p * (1 - p) / total), p + Z * np.sqrt(p * (1 - p) / total)))

In [21]:
# Ensure 'mm_yyyy' is a datetime type
cleaned_data['mm_yyyy'] = pd.to_datetime(cleaned_data['mm_yyyy'], format='%Y-%m-%d')

# Calculate the proportions of 'tipo' by month
proportion_by_month_cleaned = cleaned_data.groupby(['mm_yyyy', 'tipo']).size().unstack(fill_value=0)

# Calculate the total number of contracts per month
total_by_month_cleaned = proportion_by_month_cleaned.sum(axis=1)

# Calculate the proportion and confidence intervals by month for each type
proportion_by_month_cleaned = proportion_by_month_cleaned.div(total_by_month_cleaned, axis=0)

ci_by_month_cleaned = proportion_by_month_cleaned.apply(lambda p: proportion_ci_by_month(p, total_by_month_cleaned))

# Prepare data for LaTeX table
ci_by_month_cleaned_flat = ci_by_month_cleaned.stack().reset_index()
ci_by_month_cleaned_flat.columns = ['mm_yyyy', 'tipo', 'CI']

# Join the proportions
proportion_flat_cleaned = proportion_by_month_cleaned.stack().reset_index()
proportion_flat_cleaned.columns = ['mm_yyyy', 'tipo', 'proporcion']

# Merge proportions with confidence intervals
ci_proportion_by_month_cleaned = pd.merge(proportion_flat_cleaned, ci_by_month_cleaned_flat, on=['mm_yyyy', 'tipo'])

# Format the data for LaTeX
ci_proportion_by_month_cleaned['mm_yyyy'] = ci_proportion_by_month_cleaned['mm_yyyy'].dt.strftime('%Y-%m')

# Extract the correct confidence intervals
def extract_ci_bounds(ci_series):
    ci_lower = ci_series.apply(lambda x: x[0].values[0])
    ci_upper = ci_series.apply(lambda x: x[1].values[0])
    return ci_lower, ci_upper

# Extract lower and upper bounds
ci_lower_cleaned, ci_upper_cleaned = extract_ci_bounds(ci_proportion_by_month_cleaned['CI'])

# Add the bounds to the DataFrame
ci_proportion_by_month_cleaned['CI_lower'] = ci_lower_cleaned
ci_proportion_by_month_cleaned['CI_upper'] = ci_upper_cleaned

# Format the CI column
ci_proportion_by_month_cleaned['CI'] = ci_proportion_by_month_cleaned.apply(lambda row: f"({row['CI_lower']:.2%}, {row['CI_upper']:.2%})", axis=1)

# Select relevant columns
ci_proportion_by_month_cleaned = ci_proportion_by_month_cleaned[['mm_yyyy', 'tipo', 'proporcion', 'CI']]

In [22]:
ci_by_month_cleaned

tipo,Contrato de Obras,Contrato de Servicios,Contrato de Suministros
mm_yyyy,,,
2016-01-01,"([-0.0789806068110328, -0.139122647229289, -0....","([0.2511969573207727, 0.14702788764549196, 0.2...","([0.07935652563159407, -0.021280356632709274, ..."
2016-02-01,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.3880843972953356, 0.28946449079611114, 0.3...","([0.0547510639620023, -0.043868842537222175, 0..."
2016-03-01,"([-0.07044238862827906, -0.13963024518498762, ...","([0.20451888231379167, 0.09991667534541426, 0....","([0.08890163802064033, -0.012379013370626468, ..."
2016-04-01,"([-0.04611276584927079, -0.12680178025772726, ...","([0.009535284742372196, -0.08363636363636362, ...","([0.2511969573207727, 0.14702788764549196, 0.2..."
2016-05-01,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.15079294879378785, 0.04683843973699048, 0....","([0.261904059904899, 0.15794955084810164, 0.21..."
2016-06-01,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.33890163802064033, 0.23762098662937353, 0....","([0.08890163802064033, -0.012379013370626468, ..."
2016-07-01,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.08890163802064033, -0.012379013370626468, ...","([0.33890163802064033, 0.23762098662937353, 0...."
2016-08-01,"([-0.07044238862827906, -0.13963024518498762, ...","([-0.07044238862827906, -0.13963024518498762, ...","([0.4941058457451242, 0.4035176772185917, 0.44..."
2016-09-01,"([-0.03638489414896667, -0.12006665972366859, ...","([0.11048881321660942, 0.008000000000000007, 0...","([0.11048881321660942, 0.008000000000000007, 0..."


In [11]:
import numpy as np
from scipy import stats

# Número de observaciones
n = len(formalizacion_data)

# Valor crítico Z para un intervalo de confianza del 95%
Z = 1.96

# 1. Intervalo de confianza para la media ponderada
weighted_mean_ci = (
    weighted_mean - Z * (weighted_std_dev / np.sqrt(n)),
    weighted_mean + Z * (weighted_std_dev / np.sqrt(n))
)

# 2. Intervalo de confianza para la media aritmética simple
simple_std_dev = formalizacion_data['amount'].std()
simple_mean_ci = (
    simple_mean - Z * (simple_std_dev / np.sqrt(n)),
    simple_mean + Z * (simple_std_dev / np.sqrt(n))
)

# 3. Intervalo de confianza para la proporción
def proportion_ci(proportion, n, Z=1.96):
    return (
        proportion - Z * np.sqrt(proportion * (1 - proportion) / n),
        proportion + Z * np.sqrt(proportion * (1 - proportion) / n)
    )

proportion_cis = {tipo: proportion_ci(proportion, n) for tipo, proportion in proportion_tipo.items()}

weighted_mean_ci, simple_mean_ci, proportion_cis

((np.float64(1603039.4243845567), np.float64(3393754.93572292)),
 (np.float64(1686729.131651928), np.float64(3626794.5324232606)),
 {'Suministro': (np.float64(0.35291240743181856),
   np.float64(0.4490926050995097)),
  'Servicios': (np.float64(0.27017913219017214),
   np.float64(0.3613998151782489)),
  'Servicio': (np.float64(0.09282741758907467),
   np.float64(0.1577991488269654)),
  'Obras': (np.float64(0.0388771517563829), np.float64(0.08643613145163714)),
  'Suministros': (np.float64(0.020849125972790428),
   np.float64(0.0593513752803424)),
  'Contrato de servicios': (np.float64(0.0002498712208851145),
   np.float64(0.019800254092398092)),
  'Contrato de obras': (np.float64(-0.0009574762480308308),
   np.float64(0.015995070232993236)),
  'Contrato de Servicios': (np.float64(-0.0009574762480308308),
   np.float64(0.015995070232993236)),
  'Gestión de servicio público': (np.float64(-0.001917049726421996),
   np.float64(0.0119421123830636)),
  'Gestión de Servicios Públicos': (np.flo

In [12]:
# Calculate the proportions of 'tipo' by month
proportion_by_month = formalizacion_data.groupby(['mm_yyyy', 'tipo']).size().unstack(fill_value=0)

# Calculate the total number of contracts per month
total_by_month = proportion_by_month.sum(axis=1)

# Calculate the proportion and confidence intervals by month for each type
proportion_by_month = proportion_by_month.div(total_by_month, axis=0)

def proportion_ci_by_month(proportion, total, Z=1.96):
    return proportion.apply(lambda p: (p - Z * np.sqrt(p * (1 - p) / total), p + Z * np.sqrt(p * (1 - p) / total)))

ci_by_month = proportion_by_month.apply(lambda p: proportion_ci_by_month(p, total_by_month))

ci_by_month

tipo,Administrativo,Administrativo de servicio,Concesión del servicio público educativo,Contrato administrativo de servicio,Contrato de Servicios,Contrato de obras,Contrato de servicios,Contrato privado,Gestión de Servicios Públicos,Gestión de servicio público,Obra,Obras,Ordinario,Privado,Servicio,Servicios,Suministro,Suministro sucesivo y continuado,Suministros
mm_yyyy,,,,,,,,,,,,,,,,,,,
01-2016,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.0730464276002019, -0.06144619253655999, -...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.0441957599601816, -0.028554006191284315, ...","([0.13772189581554434, 0.1584139910862491, 0.1...","([0.06661111689694282, 0.08639654430063817, 0....","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
01-2017,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.07428804853405788, -0.06347880420374899, ...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.05513275587509045, -0.04044588494109075, ...","([0.030110425545208153, 0.04907109439030105, 0...","([0.21709836809708338, 0.2380839829258241, 0.2...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
01-2018,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.06212184087736347, -0.04824116139070414, ...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([-0.07445916362186211, -0.06429954652915759, ...","([0.1568172386839548, 0.17763825791394383, 0.1...","([0.10108167275627578, 0.12140090694168482, 0....","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
02-2016,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.